# VPR Pipeline Demo (Colab-ready)
This notebook runs a quick demo of the VPR pipeline.

## 1. Install dependencies (run in Colab)
Run the following cell to install required libraries.

In [ ]:
# ✅ Clone & install everything
!git clone https://github.com/SabraHashemi/VPR-Pipeline-Final.git
%cd VPR-Pipeline-Final
!pip install -r requirements.txt
!bash scripts/setup_colab.sh
!bash scripts/download_datasets.sh
!bash scripts/download_matcher_weights.sh

In [ ]:
! bash scripts/install_backbones.sh


## 2. (Optional) Install heavy matchers and MixVPR
If you plan to use Full mode, install MixVPR and download weights as instructed in the repo.

## 3. Run the light pipeline (fast)

In [ ]:
!python3 src/cli.py pipeline --db example_data/database --query example_data/queries --backbone resnet50 --mode light

In [ ]:
!python3 src/cli.py pipeline \
  --db datasets/GSV-XS/database \
  --query datasets/GSV-XS/queries \
  --backbone netvlad \
  --mode full \
  --k 10 \
  --use_gpu \
  --run_name "GSV-XS_NetVLAD_Full_GPU"

## 4. Inspect results

In [ ]:
import json, os
print(open('outputs/pipeline_results.json').read())
print('Matches folder:', os.listdir('matches') if os.path.exists('matches') else 'none')